##This notebook uses the embeddings created against each node. Store them in Vector Index and make a search using VectorIndex for fast retreival of data

In [1]:
!pip install langchain langchain-openai langchain_neo4j  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [2]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data

In [3]:
import os
props = load_properties("openai_key.txt")

os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
#os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph, Neo4jVector

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [5]:
embedding_provider = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_KEY")
)

In [6]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

## Neo4j allows only one vector index per (Label, Property) pair.

In [8]:
#Neo4j allows only one vector index per (Label, Property) pair.
movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="moviePlots", #this Vector Index already exists in Neo4j database
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

In [15]:
result = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.",k=5)

for doc in result:
    print(doc.metadata["title"], "-", doc.page_content)
    print(f"--------------")

GoldenEye - James Bond teams up with the lone survivor of a destroyed Russian research center to stop the hijacking of a nuclear space weapon by a fellow agent formerly believed to be dead.
--------------
From Dusk Till Dawn - Two criminals and their hostages unknowingly seek temporary refuge in an establishment populated by vampires, with chaotic results.
--------------
Toy Story - A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.
--------------
Nick of Time - An unimpressive, every-day man is forced into a situation where he is told to kill a politician to save his kidnapped daughter.
--------------
Jumanji - When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.
--------------
